1. Import polars, and load the data into Dataframes

In [1]:
import polars as pl

In [2]:
df = pl.read_parquet("data/pp_data_man.parquet")
df2 = pl.read_parquet("data/pc_man.parquet")

2. Check Dataframes

In [3]:
df.head(5)

column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""{D93B27B1-CBD4-3100-E053-6C04A…","""140,000.00""","""2022-02-16 00:00""","""SK16 4DT""","""T""","""N""","""F""","""70""","""""","""CHAPEL STREET""","""""","""DUKINFIELD""","""TAMESIDE""","""GREATER MANCHESTER"""
"""{2ACACE8D-02B4-295E-E063-4804A…","""278,000.00""","""2024-11-29 00:00""","""SK6 1QW""","""S""","""N""","""F""","""1""","""""","""BRIAR GROVE""","""WOODLEY""","""STOCKPORT""","""STOCKPORT""","""GREATER MANCHESTER"""
"""{01EB45EF-F1B0-40F3-E063-4704A…","""345,448.00""","""2022-02-11 00:00""","""M1 2EY""","""F""","""Y""","""L""","""72""","""FLAT 902""","""CHAPELTOWN STREET""","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER"""
"""{879537F9-FB6B-4663-A0E5-5E407…","""93,000.00""","""2004-09-22 00:00""","""BL8 2RR""","""T""","""N""","""L""","""43""","""""","""NEWBOLD STREET""","""BURY""","""BURY""","""BURY""","""GREATER MANCHESTER"""
"""{7011B10A-2B91-8ED6-E053-6B04A…","""80,000.00""","""2018-05-03 00:00""","""BL3 4HE""","""T""","""N""","""F""","""243""","""""","""WILLOWS LANE""","""""","""BOLTON""","""BOLTON""","""GREATER MANCHESTER"""


In [4]:
df2.head(5)

pcds,lat,long
str,f64,f64
"""AL1 3PE""",51.755864,-0.327577
"""AL1 4FJ""",51.761598,-0.326075
"""AL1 5UE""",51.751596,-0.325894
"""AL2 1NT""",51.720229,-0.295058
"""AL2 2PJ""",51.721518,-0.337343


3. Rename columns, as properties are missing them and standardize the coordinates for future joining

In [5]:
df.columns = [
    "id", "price", "date", "postcode", "property_type", "new", "duration",
    "paon", "saon", "street", "locality", "town_city", "district", "county"
]
df2 = df2.rename({"pcds": "postcode", "lat": "latitude", "long": "longitude"})

4.visual double check for the missing values/empty rows

In [6]:
df.select(pl.col("saon")).head(10)


saon
str
""""""
""""""
"""FLAT 902"""
""""""
""""""
"""APARTMENT 23"""
""""""
""""""
""""""


5.Count nulls

In [7]:
df.null_count()

id,price,date,postcode,property_type,new,duration,paon,saon,street,locality,town_city,district,county
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:

df2.null_count()

postcode,latitude,longitude
u32,u32,u32
0,0,0


6. check data types

In [9]:
df.schema

Schema([('id', String),
        ('price', String),
        ('date', String),
        ('postcode', String),
        ('property_type', String),
        ('new', String),
        ('duration', String),
        ('paon', String),
        ('saon', String),
        ('street', String),
        ('locality', String),
        ('town_city', String),
        ('district', String),
        ('county', String)])

In [10]:
df2.schema

Schema([('postcode', String), ('latitude', Float64), ('longitude', Float64)])

7. Standardize postcodes for future joining

In [11]:
df = df.with_columns(
    pl.col("postcode")
      .cast(pl.Utf8)
      .str.strip_chars()
      .str.to_uppercase()
      .str.replace_all(r"\s+", " ")
      .alias("postcode")
)

df2 = df2.with_columns(
    pl.col("postcode")
      .cast(pl.Utf8)
      .str.strip_chars()
      .str.to_uppercase()
      .str.replace_all(r"\s+", " ")
      .alias("postcode")
)

8.Join dataframes into 1

In [12]:
df = df.join(
    df2.select("postcode", "latitude", "longitude"),
    on="postcode",
    how="left"
)

9. Replace "" with proper nulls as visually we are missing data.

In [13]:
string_cols = [c for c, t in df.schema.items() if t == pl.Utf8]
df = df.with_columns([
    pl.when(pl.col(c) == "").then(None).otherwise(pl.col(c)).alias(c) for c in string_cols
    ])

10.Second null check.

In [14]:
df.null_count()


id,price,date,postcode,property_type,new,duration,paon,saon,street,locality,town_city,district,county,latitude,longitude
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,1169,0,0,0,21,1208175,1414,423423,0,0,0,1199,1199


11. Concatenate Paon and saon as some addresses dont have a secondary address object and then drop the column

In [15]:
df = df.with_columns(
    pl.when(pl.col("saon").is_not_null() & (pl.col("saon") != ""))
      .then(pl.concat_str([pl.col("paon"), pl.col("saon")], separator=", "))
      .otherwise(pl.col("paon"))
      .alias("paon")
).drop("saon")



12. check what records are missing the street.

In [ ]:
null_street = df.filter(pl.col("street").is_null())
print(null_street)


shape: (1_414, 15)
┌───────────┬───────────┬───────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ id        ┆ price     ┆ date      ┆ postcode ┆ … ┆ district  ┆ county    ┆ latitude  ┆ longitude │
│ ---       ┆ ---       ┆ ---       ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str       ┆ str       ┆ str       ┆ str      ┆   ┆ str       ┆ str       ┆ f64       ┆ f64       │
╞═══════════╪═══════════╪═══════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ {76492375 ┆ 54,000.00 ┆ 2002-06-1 ┆ M34 3LY  ┆ … ┆ TAMESIDE  ┆ GREATER   ┆ 53.460581 ┆ -2.117257 │
│ -1598-4FD ┆           ┆ 4 00:00   ┆          ┆   ┆           ┆ MANCHESTE ┆           ┆           │
│ B-827E-B5 ┆           ┆           ┆          ┆   ┆           ┆ R         ┆           ┆           │
│ A68…      ┆           ┆           ┆          ┆   ┆           ┆           ┆           ┆           │
│ {5D5B84C7 ┆ 8,000.00  ┆ 2002-03-1 ┆ M6 5SH   ┆ … ┆ SALFORD   ┆ GREATER

13. Drop rows with missing postcodes and the whole locality column as we have the district-city-county columns

In [17]:
df = df.drop_nulls(subset=["postcode"])
df = df.drop("locality")


14. make sure post code is in the right format

In [18]:
df = df.with_columns(
    pl.col("postcode")
      .str.strip_chars()
      .str.to_uppercase()
      .str.replace_all(r"\s+", " ")
      .alias("postcode")
)



15. check whats next in the cleaning docket

In [19]:
df.head(5)

id,price,date,postcode,property_type,new,duration,paon,street,town_city,district,county,latitude,longitude
str,str,str,str,str,str,str,str,str,str,str,str,f64,f64
"""{D93B27B1-CBD4-3100-E053-6C04A…","""140,000.00""","""2022-02-16 00:00""","""SK16 4DT""","""T""","""N""","""F""","""70""","""CHAPEL STREET""","""DUKINFIELD""","""TAMESIDE""","""GREATER MANCHESTER""",53.477466,-2.091735
"""{2ACACE8D-02B4-295E-E063-4804A…","""278,000.00""","""2024-11-29 00:00""","""SK6 1QW""","""S""","""N""","""F""","""1""","""BRIAR GROVE""","""STOCKPORT""","""STOCKPORT""","""GREATER MANCHESTER""",53.427355,-2.102642
"""{01EB45EF-F1B0-40F3-E063-4704A…","""345,448.00""","""2022-02-11 00:00""","""M1 2EY""","""F""","""Y""","""L""","""72, FLAT 902""","""CHAPELTOWN STREET""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER""",53.478898,-2.224708
"""{879537F9-FB6B-4663-A0E5-5E407…","""93,000.00""","""2004-09-22 00:00""","""BL8 2RR""","""T""","""N""","""L""","""43""","""NEWBOLD STREET""","""BURY""","""BURY""","""GREATER MANCHESTER""",53.594024,-2.318071
"""{7011B10A-2B91-8ED6-E053-6B04A…","""80,000.00""","""2018-05-03 00:00""","""BL3 4HE""","""T""","""N""","""F""","""243""","""WILLOWS LANE""","""BOLTON""","""BOLTON""","""GREATER MANCHESTER""",53.56682,-2.458227


16. remove the hours from the date as its not needed for a cleaner date and it will interfere with the analysis further down

In [20]:
df = df.with_columns(
    pl.col("date").str.slice(0, 10).str.strptime(pl.Date, format="%Y-%m-%d", strict=True).alias("date")
)



17. change the price to a float

In [21]:
df = df.with_columns(
    pl.col("price")
      .str.replace_all(",", "")
      .cast(pl.Float64)
      .alias("price")
)



18.Separate the date into individual yyyy/mm/dd columns for easier analysis

In [22]:
df = df.with_columns([
    pl.col("date").dt.year().alias("sale_year"),
    pl.col("date").dt.month().alias("sale_month"),
    pl.col("date").dt.quarter().alias("sale_quarter")
])

19.Prepare the street column to fill the missing values by filling missing values with the street from the same postcode

In [24]:
df_non_null_street = df.filter(pl.col("street").is_not_null())

In [25]:
postcode_mode_list = (
    df_non_null_street
    .group_by("postcode")
    .agg(
        pl.col("street").mode().alias("street_modes")
    )
)

In [26]:
postcode_fill = postcode_mode_list.with_columns(
    pl.col("street_modes").list.first().alias("imputed_street")
).drop("street_modes")

21. Fill empty streets with the common street for the postcode

In [27]:
df_joined = df.join(postcode_fill, on="postcode", how="left")

In [28]:
df_filled = df_joined.with_columns(
    pl.when(pl.col("street").is_null())
      .then(pl.col("imputed_street"))
      .otherwise(pl.col("street"))
      .alias("street")
).drop("imputed_street")

22. check the new null count

In [29]:
df_filled.null_count()

id,price,date,postcode,property_type,new,duration,paon,street,town_city,district,county,latitude,longitude,sale_year,sale_month,sale_quarter
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,21,620,0,0,0,30,30,0,0,0


23. fill missing street and primary address object with not applicable

In [30]:
df_filled = df_filled.with_columns([
    pl.col("paon").fill_null("N/A"),
    pl.col("street").fill_null("N/A")
])

24. Standardize and change data type for the category columns

In [31]:
categorical_cols = ["property_type", "duration", "new"]

for col in categorical_cols:
    if col in df_filled.columns:
        df_filled = df_filled.with_columns(
            pl.col(col)
            .str.strip_chars()
            .str.to_lowercase()
            .cast(pl.Categorical)
            .alias(col)
        )

25. final data check before analysis.

In [32]:
df_filled.head(5)

id,price,date,postcode,property_type,new,duration,paon,street,town_city,district,county,latitude,longitude,sale_year,sale_month,sale_quarter
str,f64,date,str,cat,cat,cat,str,str,str,str,str,f64,f64,i32,i8,i8
"""{D93B27B1-CBD4-3100-E053-6C04A…",140000.0,2022-02-16,"""SK16 4DT""","""t""","""n""","""f""","""70""","""CHAPEL STREET""","""DUKINFIELD""","""TAMESIDE""","""GREATER MANCHESTER""",53.477466,-2.091735,2022,2,1
"""{2ACACE8D-02B4-295E-E063-4804A…",278000.0,2024-11-29,"""SK6 1QW""","""s""","""n""","""f""","""1""","""BRIAR GROVE""","""STOCKPORT""","""STOCKPORT""","""GREATER MANCHESTER""",53.427355,-2.102642,2024,11,4
"""{01EB45EF-F1B0-40F3-E063-4704A…",345448.0,2022-02-11,"""M1 2EY""","""f""","""y""","""l""","""72, FLAT 902""","""CHAPELTOWN STREET""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER""",53.478898,-2.224708,2022,2,1
"""{879537F9-FB6B-4663-A0E5-5E407…",93000.0,2004-09-22,"""BL8 2RR""","""t""","""n""","""l""","""43""","""NEWBOLD STREET""","""BURY""","""BURY""","""GREATER MANCHESTER""",53.594024,-2.318071,2004,9,3
"""{7011B10A-2B91-8ED6-E053-6B04A…",80000.0,2018-05-03,"""BL3 4HE""","""t""","""n""","""f""","""243""","""WILLOWS LANE""","""BOLTON""","""BOLTON""","""GREATER MANCHESTER""",53.56682,-2.458227,2018,5,2


In [33]:
df_filled.schema

Schema([('id', String),
        ('price', Float64),
        ('date', Date),
        ('postcode', String),
        ('property_type', Categorical),
        ('new', Categorical),
        ('duration', Categorical),
        ('paon', String),
        ('street', String),
        ('town_city', String),
        ('district', String),
        ('county', String),
        ('latitude', Float64),
        ('longitude', Float64),
        ('sale_year', Int32),
        ('sale_month', Int8),
        ('sale_quarter', Int8)])

26. Looks good ready for pipeline implementation